In [126]:
# !pip install tensorflow
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from collections import deque
import random
from qnetwork import DQNetwork
from dataset import Cifar10ImageDataset
from agent import DQNAgent

In [127]:
class DQNetwork:
    def __init__(self, state_size, action_size, gamma: float, epsilon: float, learning_rate: float):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma # Discount factor
        self.epsilon = epsilon  # Exploration-exploitation trade-off
        self.learning_rate = learning_rate
        
        print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
        
        # Main Q-network
        self.model = self.build_network()

        # Target Q-network
        self.target_model = self.build_network()
        
        self.update_target_network()
    
    def update_target_network(self):
        self.target_model.set_weights(self.model.get_weights())

    def build_network(self):
        # Input layer of the network
        input_layer = tf.keras.layers.Input(shape = self.state_size)

        # Convolution Layers
        conv2 = tf.keras.layers.Conv2D(32, 5, strides=2, activation=tf.nn.relu)(input_layer)
        conv2 = tf.keras.layers.Conv2D(32, 5, strides=2, activation=tf.nn.relu)(conv2)
        flatten = tf.keras.layers.Flatten()(conv2)
        outputs = tf.keras.layers.Dense(self.action_size, activation="softmax")(flatten)
        model = tf.keras.Model(inputs=input_layer, outputs=outputs)
        
        # Compiling Deep Q Network Model
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate), loss='categorical_crossentropy')
        
        return model

In [196]:
import pickle
import datetime
import os
import random

from collections import deque

import numpy as np
import tensorflow as tf
from sklearn import metrics
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


class DQNAgent:
    def __init__(self, network, dataset, state_size, action_size, memory, epsilon):
        self.network = network
        self.dataset = dataset
        self.memory = memory
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=1024)
        
        self.epsilon = epsilon
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01

    def remember(self, state, action, reward, next_state, terminal):
        self.memory.append((state, action, reward, next_state, terminal))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.action_size)
        q_values = self.network.model.predict(state)
        return np.argmax(q_values[0])
    
    def get_reward_and_terminal(self, label, action):
        terminal = 0
        if action == label:
            reward = self.dataset.reward_set[label]
        else:
            reward = - self.dataset.reward_set[label]
            # End of an episode if the agent misjudgement about a minority class
            if label in self.dataset.minority_classes:
                terminal = 1
        return reward, terminal
    
    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return

        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, terminal in minibatch:
            target = self.network.model.predict(state)
            if terminal:
                target[0][action] = reward
            else:
                t = self.network.target_model.predict(next_state)[0]
                target[0][action] = reward + self.network.gamma * np.amax(t)

            print("Target is ", target)
            self.network.model.fit(state, target, epochs=1, verbose=0)

        self.update_epsilon()

    def update_epsilon(self):
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def save_model(self, save_path):
        self.network.model.save_weights(save_path)

    def train(self, num_episodes):
        for episode in range(num_episodes):
            int_val = np.random.randint(0, self.dataset.length_of_dataset)
            state_batch = self.dataset.dataset.take(int_val)
            for state, label in list(state_batch):
                action = self.act(state)
                next_state = self.dataset.dataset.take(np.random.randint(0, self.dataset.length_of_dataset))
                next_state = tuple(next_state)[0][0]
                
                print(f"Action is {action} and label is {label}")
                reward, terminal = self.get_reward_and_terminal(label, action)
                self.remember(state, action, reward, next_state, terminal)
                state = next_state

                if terminal == 1:
                    self.network.update_target_network()
                    print("Episode: {}, Reward: {}".format(episode, reward))
                    break

                if len(self.memory) > self.dataset.batch_size:
                    self.replay(self.dataset.batch_size)
        # for episode in range(num_episodes):
        #     int_val = np.random.randint(0, len(self.dataset.training_data_batches))
        #     state_batch = self.dataset.training_data_batches.take(int_val)
        #     for states, labels in tuple(state_batch):
        #         for index, val in enumerate(labels):
        #             state = states[index]
        #             action = self.act(state.numpy())
        #             next_state = states[np.random.randint(0, len(states))]
                    
        #             print(f"Action is {action} and label is {labels[index]}")
        #             reward, terminal = self.get_reward_and_terminal(labels[index], action)
        #             self.remember(state, action, reward, next_state, terminal)
        #             state = next_state

        #             if terminal == 1:
        #                 self.network.update_target_network()
        #                 print("Episode: {}, Reward: {}".format(episode, reward))
        #                 break

        #             if len(self.memory) > self.dataset.batch_size:
        #                 self.replay(self.dataset.batch_size)
                
        # Testing the model
        # total_reward = 0
        # for test_state in X_test:
        #     test_state = np.reshape(test_state, [1, state_size])
        #     action = self.act(test_state)
        #     reward = 1 if y_test[np.argmax(test_state)] == 1 else -1
        #     total_reward += reward

        # print("Test Accuracy: {:.2%}".format(total_reward / len(X_test)))
        
    def evaluate(self, train_label, train_prediction, val_label, val_prediction, step, show_phase="Both"):
        # Calculate f1 score of each class and weighted macro average
        print("train_step : {}, epsilon : {:.3f}".format(step, self.epsilon))
        if show_phase == "Both":
            phase = ["Train Data.", "Validation Data."]
            labels = [train_label, val_label]
            predictions = [train_prediction, val_prediction]
        elif show_phase == "Train":
            phase = ["Train Data."]
            labels = [train_label]
            predictions = [train_prediction]
        elif show_phase == "Validation":
            phase = ["Validation Data."]
            labels = [val_label]
            predictions = [val_prediction]

        for idx, (label, prediction) in enumerate(zip(labels, predictions)):
            f1_all_cls = metrics.f1_score(label, prediction, average=None)
            f1_macro_avg = metrics.f1_score(label, prediction, average='weighted')
            print("\t\t {:<20} f1-score of ".format(phase[idx]), end="")
            for i, f1 in enumerate(f1_all_cls):
                print("class {} : {:.3f}".format(i, f1), end=", ")
            print("weighted macro avg : {:.3f}".format(f1_macro_avg))

In [195]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

class Cifar10ImageDataset:
    def __init__(self, batch_size=32):

        self.batch_size = batch_size
        self.create_dataset()
        self.get_labels_counts()
        self.get_rho()
        self.get_minority_classes()

    def create_dataset(self):
        (X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
        
        X = np.concatenate([X_train, X_test])
        y = np.concatenate([y_train, y_test])
        
        # Convert labels to integers
        y = y.flatten()

        # Create a TensorFlow Dataset from the CIFAR-10 data
        dataset = tf.data.Dataset.from_tensor_slices((X, y))
        
        # We are going to get 25% minority classes from total classes i-e if there are total 6 classes then we will only set 2 classes as minority classes
        self.no_of_minority_classes_to_get = int(np.round(len(np.unique(y)) * 0.25))
        
        
        # Specify the percentage of label 2 data to remove
        percentage_to_remove = 0.9
        for min_ in range(self.no_of_minority_classes_to_get):
            # Use the filter_data function to create a new dataset with filtered data
            dataset = dataset.filter(lambda x, y: tf.py_function(filter_data, inp=[x, y, min_, percentage_to_remove], Tout=tf.bool))
            percentage_to_remove -= 0.1
        
        self.dataset = dataset
        
        self.length_of_dataset = len(list(self.dataset.as_numpy_iterator()))
    
    # Define a function to filter out data with label 2 based on a percentage
    def filter_data(self, image, label, label_to_drop, percentage_to_remove):
        # Assuming label 2 corresponds to the class you want to remove
        if label == label_to_drop.numpy() and tf.random.uniform(()) < percentage_to_remove:
            return False
        return True
    
    def get_labels_counts(self):
        labels_dataset = self.dataset.map(lambda x, y: y)
        
        # Convert the labels dataset to a NumPy array
        labels_array = list(labels_dataset.as_numpy_iterator())

        # Get unique labels and their counts
        self.unique_labels, self.label_counts = np.unique(labels_array, return_counts=True)
        
        return label_counts
        
    def get_minority_classes(self):
        unique_labels_counts_dict = dict(zip(self.unique_labels, self.label_counts))
        unique_labels_counts_dict = sorted(unique_labels_counts_dict.items())
        
        self.minority_classes = []
        for i in range(self.no_of_minority_classes_to_get):
            self.minority_classes.append(unique_labels_counts_dict[i][0])

    def get_rho(self):
        """
        In the two-class dataset problem, this paper has proven that the best performance is achieved when the reciprocal of the ratio of the number of data is used as the reward function.
        In this code, the result of this paper is extended to multi-class by creating a reward function with the reciprocal of the number of data for each class.
        """
        labels_counts = self.get_labels_counts()
        raw_reward_set = 1 / labels_counts
        self.reward_set = np.round(raw_reward_set / np.linalg.norm(raw_reward_set), 6)
        print("\nReward for each class.")
        for cl_idx, cl_reward in enumerate(self.reward_set):
            print("\t- Class {} : {:.6f}".format(cl_idx, cl_reward))

In [2]:
def reading_csv(folder_path: str, file_path: str):
    df = pd.read_csv(file_path)
    df = shuffle(df, random_state=42)
    df["filepath"] = folder_path + df["image_id"]
    return df

In [4]:
df = pd.read_csv("../digit-recognizer/train.csv")

2

In [179]:
# Load the CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

train_images = np.concatenate([X_train, X_test])
train_labels = np.concatenate([y_train, y_test])
# Convert labels to integers
train_labels = train_labels.flatten()

# Create a TensorFlow Dataset from the CIFAR-10 data
cifar_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))

# Define a function to filter out data with label 2 based on a percentage
def filter_data(image, label, label_to_drop, percentage_to_remove):
    # Assuming label 2 corresponds to the class you want to remove
    if label == label_to_drop.numpy() and tf.random.uniform(()) < percentage_to_remove:
        return False
    return True

# Specify the percentage of label 2 data to remove
percentage_to_remove = 0.9 # Example: Remove 30% of label 2 data

# Use the filter_data function to create a new dataset with filtered data
filtered_dataset = cifar_dataset.filter(lambda x, y: tf.py_function(filter_data, inp=[x, y, 2, percentage_to_remove], Tout=tf.bool))
# Use the filter_data function to create a new dataset with filtered data
filtered_dataset = filtered_dataset.filter(lambda x, y: tf.py_function(filter_data, inp=[x, y, 3, percentage_to_remove], Tout=tf.bool))


In [193]:
filtered_dataset

<_FilterDataset element_spec=(TensorSpec(shape=(32, 32, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.uint8, name=None))>

In [194]:
len(list(filtered_dataset.as_numpy_iterator()))
filtered_dataset.reduce(0, lambda x, _: x + 1).numpy()

49255

In [144]:
labels_dataset = filtered_dataset.map(lambda x, y: y)

# Convert the labels dataset to a NumPy array
labels_array = list(labels_dataset.as_numpy_iterator())

# Get unique labels and their counts
unique_labels, label_counts = np.unique(labels_array, return_counts=True)

# Create a dictionary mapping unique labels to their counts
label_count_dict = dict(zip(unique_labels, label_counts))

# Print the unique labels and their counts
for label, count in label_count_dict.items():
    print(f"Label {label}: Count {count}")

Label 0: Count 5000
Label 1: Count 5000
Label 2: Count 473
Label 3: Count 487
Label 4: Count 5000
Label 5: Count 5000
Label 6: Count 5000
Label 7: Count 5000
Label 8: Count 5000
Label 9: Count 5000


In [93]:
np.unique(train_labels, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8),
 array([5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000],
       dtype=int64))

In [99]:
list(filtered_dataset.as_numpy_iterator())[0]

(array([[[ 59,  62,  63],
         [ 43,  46,  45],
         [ 50,  48,  43],
         ...,
         [158, 132, 108],
         [152, 125, 102],
         [148, 124, 103]],
 
        [[ 16,  20,  20],
         [  0,   0,   0],
         [ 18,   8,   0],
         ...,
         [123,  88,  55],
         [119,  83,  50],
         [122,  87,  57]],
 
        [[ 25,  24,  21],
         [ 16,   7,   0],
         [ 49,  27,   8],
         ...,
         [118,  84,  50],
         [120,  84,  50],
         [109,  73,  42]],
 
        ...,
 
        [[208, 170,  96],
         [201, 153,  34],
         [198, 161,  26],
         ...,
         [160, 133,  70],
         [ 56,  31,   7],
         [ 53,  34,  20]],
 
        [[180, 139,  96],
         [173, 123,  42],
         [186, 144,  30],
         ...,
         [184, 148,  94],
         [ 97,  62,  34],
         [ 83,  53,  34]],
 
        [[177, 144, 116],
         [168, 129,  94],
         [179, 142,  87],
         ...,
         [216, 184, 140],
  

In [87]:
print("Number of samples in the filtered dataset:", len(list(filtered_dataset.as_numpy_iterator())))

Number of samples in the filtered dataset: 45967


In [73]:
# Specify the percentage of label 2 data to remove
percentage_to_remove = 0.8

# Use the filter_data function to create a new dataset with filtered data
filtered_dataset = cifar_dataset.filter(lambda x, y: tf.py_function(filter_data, inp=[x, y, 2, percentage_to_remove], Tout=tf.bool))

In [90]:
for img, label in filtered_dataset.take(1):
    print(img.shape)
    print(label)

(32, 32, 3)
tf.Tensor(6, shape=(), dtype=uint8)


In [ ]:
import tensorflow as tf

# Load the CIFAR-10 dataset
(train_images, train_labels), (_, _) = tf.keras.datasets.cifar10.load_data()

# Convert labels to integers
train_labels = train_labels.flatten()

# Create a TensorFlow Dataset from the CIFAR-10 data
cifar_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))

# Define a function to filter out data with label 2 based on a percentage
def filter_data(image, label, percentage_to_remove):
    # Assuming label 2 corresponds to the class you want to remove
    if label == 2 and tf.random.uniform(()) < percentage_to_remove:
        return False
    return True

# Specify the percentage of label 2 data to remove
percentage_to_remove = 0.3  # Example: Remove 30% of label 2 data

# Use the filter_data function to create a new dataset with filtered data
filtered_dataset = cifar_dataset.filter(lambda x, y: tf.py_function(filter_data, inp=[x, y, percentage_to_remove], Tout=tf.bool))

# Print the number of samples in the filtered dataset
print("Number of samples in the filtered dataset:", len(list(filtered_dataset.as_numpy_iterator())))

# Optionally, iterate over the filtered dataset to inspect the data
for image, label in filtered_dataset.take(5):
    print("Label:", label.numpy())
    # Additional processing or visualization code can be added here

In [ ]:
def drop_some_labels(X_train, y_train):
    for train_data in X_train

In [4]:
# df = reading_csv("../cassava-leaf-disease-classification/train_images/", "../cassava-leaf-disease-classification/train.csv")

In [5]:
def drop_some_labels(df, label):
    label_ = df[df['label'] == label]
    n_to_drop = label_.shape[0]//3
    index_to_drop = label_.sample(n_to_drop).index
    df = df.drop(index_to_drop)
    return df

In [6]:
df = drop_some_labels(df, 2)

In [197]:
dataset = Cifar10ImageDataset(32)


Reward for each class.
	- Class 0 : 0.066644
	- Class 1 : 0.066644
	- Class 2 : 0.704483
	- Class 3 : 0.684231
	- Class 4 : 0.066644
	- Class 5 : 0.066644
	- Class 6 : 0.066644
	- Class 7 : 0.066644
	- Class 8 : 0.066644
	- Class 9 : 0.066644


In [198]:
network = DQNetwork(state_size=(32, 32, 3), action_size=10, gamma = 0.95, epsilon = 1.0, learning_rate = 0.001)

Num GPUs Available:  0


In [199]:
agent = DQNAgent(network, dataset, state_size=(32, 32, 3), action_size = 10, memory=deque(maxlen=2000), epsilon=1.0)

In [ ]:
agent.train(10)

Action is 7 and label is 6
Action is 6 and label is 9
Action is 6 and label is 9
Action is 5 and label is 4
Action is 2 and label is 1
Episode: 0, Reward: -0.066644
Action is 1 and label is 6
Action is 9 and label is 9
Action is 0 and label is 9
Action is 9 and label is 4
Action is 0 and label is 2
Action is 2 and label is 7
Action is 9 and label is 8
Action is 9 and label is 3
Action is 8 and label is 4
Action is 7 and label is 7
Action is 3 and label is 7
Action is 5 and label is 2
Action is 8 and label is 9
Action is 0 and label is 9
Action is 2 and label is 9
Action is 2 and label is 3
Action is 7 and label is 2
Action is 6 and label is 6
Action is 9 and label is 4
Action is 4 and label is 3
